In [1]:
import os

In [2]:
%pwd

'/Users/vikashinnamuri/Desktop/Chicken_disease_classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/vikashinnamuri/Desktop/Chicken_disease_classification'

Entity is an return type of an function and what is type of an  funtion

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True) #this code  in Python is used to define a data class where instances are immutable
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [12]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories   # used to read the yaml file and create the dict

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        param_filepath=PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params=read_yaml(param_filepath)
        
        create_directories([self.config.artifacts_root],verbose=True)
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir],verbose=True)
        
        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [14]:
import os 
import  urllib.request as request  #used to download the file from the url
import zipfile  # used for unzip file 
from src.cnnClassifier import logger  #used to enter the loggers into logs folders
from src.cnnClassifier.utils.common import get_size # this gives the size of the file 

In [26]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,header=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with the info of:\n {header}")
        else:
            logger.info(f"The file already exisit with the size of{get_size(Path(self.config.local_data_file))}")
            
    def extract_unzip_file(self):
        """_summary_
        
            zip_file_path:str
            extract the zip file data into the data dictornary 
            functio return none
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            
          
                

In [31]:
#pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_unzip_file()
except Exception as e:
    raise e 
    
    

[2024-08-27 12:42:40,710: INFO: common:  yaml file:config/config.yaml successfully loaded]
[2024-08-27 12:42:40,713: INFO: common:  yaml file:params.yaml successfully loaded]
[2024-08-27 12:42:40,713: INFO: common: Created directories at: artifacts]
[2024-08-27 12:42:40,714: INFO: common: Created directories at: artifacts/data_ingestion]
[2024-08-27 12:42:51,595: INFO: 1878931494: artifacts/data_ingestion/data.zip download! with the info of:
 Connection: close
Content-Length: 11796101
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1b7ec9a8532f79337dbb964fdff0c684cd581eda3c38637a7a48630d3169b282"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E196:9F5AF:7EBA2:A880B:66CD76ED
Accept-Ranges: bytes
Date: Tue, 27 Aug 2024 07:12:41 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100029-HYD
X-Cache: 